In [ ]:
#https://github.com/sambit9238/Machine-Learning
#https://towardsdatascience.com/a-production-ready-multi-class-text-classifier-96490408757

In [81]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier

In [82]:
path = "/Users/KumarSanjeev/Downloads/ML Training Data/question_topic.csv"


In [107]:
df = pd.read_csv(path)
print(df.head(20))

b = df.iloc[0]["question_text"]
#b

# for i in range(0,50):
#     print (df.iloc[i]["question_text"],"\n")

    Unnamed: 0                                      question_text  \
0            0  Hi! If I sign up for your email list, can I se...   
1            1  I'm going to be out of the country for about a...   
2            2  I was wondering if you'd be able to overnight ...   
3            3  The Swingline electronic stapler (472555) look...   
4            4  I think this cosmetic bag would work great for...   
5            5  I'm going to be out of the state for about a w...   
6            6  I'm going to be out of the state for about a w...   
7            7  The Stanley Bostitch electronic stapler (63460...   
8            8  When is the estimated delivery date if I was t...   
9            9  I was wondering if you'd be able to overnight ...   
10          10  If I sign up for your email list, can I select...   
11          11  If I were to get this OMP driver footrest (mpn...   
12          12  Do you have any promo codes I'd be able to use...   
13          13  I'm going to be ou

In [55]:

df.shape

(5000, 3)

In [56]:
a = Counter(df["question_topic"])
a

Counter({'Sales/Promotions': 505,
         'Shipping': 799,
         'Product Availability': 833,
         'Product Specifications': 839,
         'Omnichannel': 450,
         'Product Comparison': 806,
         'Returns & Refunds': 768})

In [57]:

set(df["question_topic"])

{'Omnichannel',
 'Product Availability',
 'Product Comparison',
 'Product Specifications',
 'Returns & Refunds',
 'Sales/Promotions',
 'Shipping'}

In [58]:
#pre-processing
import re 
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [59]:
df.columns

Index(['Unnamed: 0', 'question_text', 'question_topic'], dtype='object')

In [60]:
df.shape[0]

5000

In [61]:
#train test split
from sklearn.model_selection import train_test_split
X = []
for i in range(df.shape[0]):
    X.append(clean_str(df.iloc[i][1]))
y = np.array(df["question_topic"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)


In [62]:
#feature engineering and model selection

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [63]:
#pipeline of feature engineering and model
model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [64]:
#paramater selection
from sklearn.grid_search import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

In [65]:
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X, y)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

0.9646
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 2)}


In [66]:
#fit model with training data
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...lti_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1))])

In [67]:
#evaluation on test data
pred = model.predict(X_test)

In [68]:
model.classes_

array(['Omnichannel', 'Product Availability', 'Product Comparison',
       'Product Specifications', 'Returns & Refunds', 'Sales/Promotions',
       'Shipping'], dtype='<U22')

In [69]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)

array([[128,   0,   0,   0,   0,   0,   0],
       [  0, 249,   0,   4,   0,   5,   0],
       [  0,   2, 223,   5,   0,   0,   0],
       [  0,   1,   6, 252,   0,   0,   1],
       [  0,   0,   0,   0, 230,   1,   0],
       [  0,   1,   0,   0,   0, 147,   0],
       [  2,   0,   0,   0,   0,   0, 243]])

In [70]:
accuracy_score(y_test, pred)

0.9813333333333333

In [75]:
#save the model
from sklearn.externals import joblib
joblib.dump(model, 'model_question_topic.pkl', compress=1)

['model_question_topic1.pkl']

# Deploy the model

In [77]:
from sklearn.externals import joblib
model = joblib.load('model_question_topic.pkl')

In [78]:
question = input()

I want a laptop.


In [79]:
model.predict([question])[0]

'Product Comparison'